# MODULE IMPORTS AND INSTALLATIONS

In [222]:
# !pip install yfinance requests-cache nltk python-dotenv alpaca-py matplotlib tensorflow

In [1]:
# Import packages
import dotenv
import requests
import os
import yfinance as yf
import pandas as pd
import alpaca
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [20]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=USD&interval=5min&apikey=1K8Q5IV0JSM1CB70&datatype=csv'
r = requests.get(url)
data = r.text

print(data)

timestamp,open,high,low,close,volume
2024-05-03 20:00:00,89.9700,89.9700,89.9700,89.9700,1870
2024-05-03 19:55:00,90.3900,90.5000,90.3900,90.5000,150
2024-05-03 19:40:00,90.5000,90.5000,90.5000,90.5000,2
2024-05-03 19:25:00,90.5000,90.5000,90.5000,90.5000,400
2024-05-03 19:20:00,90.0800,90.3800,90.0800,90.2500,483
2024-05-03 18:55:00,90.0000,90.0900,90.0000,90.0900,10
2024-05-03 18:45:00,89.8100,89.8100,89.8100,89.8100,10
2024-05-03 18:30:00,89.9700,89.9700,89.9700,89.9700,1870
2024-05-03 18:15:00,89.8100,89.9700,89.8100,89.8100,20
2024-05-03 18:05:00,89.9700,89.9700,89.8000,89.8000,1010
2024-05-03 18:00:00,90.0000,90.0000,90.0000,90.0000,10
2024-05-03 17:55:00,89.9700,89.9700,89.8000,89.9700,225
2024-05-03 17:50:00,89.9700,89.9700,89.9700,89.9700,1
2024-05-03 17:35:00,89.9700,89.9700,89.9700,89.9700,8
2024-05-03 17:30:00,88.1300,89.8000,87.6000,89.8000,15
2024-05-03 17:25:00,89.9700,89.9700,89.9700,89.9700,19
2024-05-03 17:20:00,89.9700,89.9700,89.9700,89.9700,16
2024-05-03 17:15:00,8

# CODE SETUP

In [2]:
# SETUP ENVIRONMENT VARIABLES
dotenv.load_dotenv()

True

# FUNCTION DEFINATIONS

In [18]:
# import nltk
# resources: punkt, averaged_perceptron_tagger, vader_lexicon, maxent_ne_chunker, words

# Initialize the sentiment analyzer
# nltk.download('words')

# FINANCIAL DATA (YFINANCE)

In [211]:
msft = yf.Ticker("BTC")
hist = msft.history(period="10y")

# Turn columns to lowercases
for hist_column in hist.columns:
    hist[hist_column.lower().replace(' ', '_')] = hist.pop(hist_column)
hist

,open,high,low,close,volume,dividends,stock_splits,capital_gains
Date,,,,,,,,
2020-10-01 00:00:00-04:00,95.702156,95.702156,95.702156,95.702156,0,0.0,0.0,0.0
2020-10-02 00:00:00-04:00,95.702156,95.702156,95.702156,95.702156,0,0.0,0.0,0.0
2020-10-05 00:00:00-04:00,95.730857,95.740429,95.697365,95.697365,10725,0.0,0.0,0.0
2020-10-06 00:00:00-04:00,95.711713,95.730850,95.711713,95.730850,10080,0.0,0.0,0.0
2020-10-07 00:00:00-04:00,95.654312,95.663884,95.625603,95.625603,6350,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-04-24 00:00:00-04:00,90.480103,90.599197,90.480103,90.599197,9183,0.0,0.0,0.0
2024-04-25 00:00:00-04:00,90.360001,90.375702,90.330002,90.360001,3428,0.0,0.0,0.0
2024-04-26 00:00:00-04:00,90.533798,90.533798,90.533798,90.533798,97,0.0,0.0,0.0


# DOWNLOADED DATA

In [6]:
# READING DOWNLOADED DATA
hist = pd.read_csv("AAPL_1min_firstratedata.csv")
hist.index = pd.to_datetime(hist["timestamp"])

# CLEAN THE DATA
hist = hist[hist.columns[1:]].dropna().drop_duplicates()
hist

,open,high,low,close,volume
timestamp,,,,,
2022-09-30 04:00:00,143.59,143.59,143.10,143.49,4060
2022-09-30 04:01:00,143.49,143.49,143.38,143.38,481
2022-09-30 04:02:00,143.22,143.22,143.22,143.22,107
2022-09-30 04:03:00,143.39,143.39,143.21,143.23,2811
2022-09-30 04:04:00,143.30,143.32,143.30,143.32,309
...,...,...,...,...,...
2023-09-29 19:45:00,171.20,171.20,171.20,171.20,800
2023-09-29 19:46:00,171.20,171.20,171.20,171.20,1917
2023-09-29 19:47:00,171.20,171.21,171.20,171.21,594


# TECHNICAL INDICATORS

In [8]:
technical_data = hist.copy()

def calculate_rsi(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def calculate_sma(data, window=14):
    return data['close'].rolling(window=window).mean()


def calculate_ema(data, span=14):
    return data['close'].ewm(span=span, adjust=False).mean()


def calculate_stochastic_oscillator(data, window=14):
    lowest_low = data['low'].rolling(window=window).min()
    highest_high = data['high'].rolling(window=window).max()
    stochastic_k = ((data['close'] - lowest_low) / (highest_high - lowest_low)) * 100
    return stochastic_k


def calculate_bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['close'].rolling(window=window).mean()
    rolling_std = data['close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band


def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = calculate_ema(data, span=short_window)
    long_ema = calculate_ema(data, span=long_window)
    macd_line = short_ema - long_ema
    signal_line = macd_line.rolling(window=signal_window).mean()
    return macd_line, signal_line


# INDICATOR WINDOWS
MA_WINDOW = 14
BOLLINGER_WINDOW = 20
MACD_SIGNAL_WINDOW = 9
MACD_SHORT_WINDOW = 12
MACD_LONG_WINDOW = 26
BOLLINGER_N_STD = 2


upper_band, lower_band = calculate_bollinger_bands(technical_data, window=BOLLINGER_WINDOW, num_std=BOLLINGER_N_STD)
technical_data['bollinger_upper'] = upper_band
technical_data['bollinger_lower'] = lower_band
technical_data['sma'] = calculate_sma(technical_data, window=MA_WINDOW)
technical_data['ema'] = calculate_ema(technical_data, span=MA_WINDOW)
technical_data['rsi'] = calculate_rsi(technical_data, window=MA_WINDOW)
technical_data['stochastic'] = calculate_stochastic_oscillator(technical_data, window=MA_WINDOW)

macd_line, signal_line = calculate_macd(technical_data, 
                                        short_window=MACD_SHORT_WINDOW,
                                        long_window=MACD_LONG_WINDOW,
                                        signal_window=MACD_SIGNAL_WINDOW)
technical_data['macd'] = macd_line
technical_data['macd_signal'] = signal_line
technical_data["day_of_week"] = technical_data.index.day_of_week
technical_data = technical_data.dropna()

# OUTCOMES OF THE TICKER
technical_data["buy_sell"] = (technical_data.open - technical_data.close > 0).apply(lambda x: 1 if x > 0 else 0)
technical_data

,open,high,low,close,volume,bollinger_upper,bollinger_lower,sma,ema,rsi,stochastic,macd,macd_signal,day_of_week,buy_sell
timestamp,,,,,,,,,,,,,,,
2022-09-30 04:27:00,143.30,143.30,143.30,143.30,200,143.509870,143.048130,143.267143,143.310117,56.410256,61.224490,-0.023125,-0.039314,4,0
2022-09-30 04:30:00,143.30,143.30,143.22,143.22,632,143.475008,143.055992,143.282857,143.298101,66.666667,42.553191,-0.028519,-0.035728,4,1
2022-09-30 04:32:00,143.30,143.30,143.30,143.30,100,143.464765,143.058235,143.302857,143.298354,69.444444,47.222222,-0.026039,-0.032777,4,0
2022-09-30 04:33:00,143.24,143.24,143.23,143.23,1121,143.464884,143.059116,143.305000,143.289240,52.459016,10.344828,-0.029383,-0.030800,4,1
2022-09-30 04:36:00,143.24,143.24,143.10,143.20,6283,143.464819,143.056181,143.302143,143.277342,46.666667,25.641026,-0.034061,-0.029876,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-29 19:45:00,171.20,171.20,171.20,171.20,800,171.351791,171.159569,171.252143,171.245882,44.434570,0.000000,-0.057752,-0.056574,4,0
2023-09-29 19:46:00,171.20,171.20,171.20,171.20,1917,171.349758,171.153252,171.248571,171.239764,44.444444,0.000000,-0.056455,-0.056573,4,0
2023-09-29 19:47:00,171.20,171.21,171.20,171.21,594,171.346968,171.149042,171.238571,171.235796,30.555556,7.142857,-0.053999,-0.057081,4,0


In [14]:
technical_data[technical_data.buy_sell == 1]

,open,high,low,close,volume,bollinger_upper,bollinger_lower,sma,ema,rsi,stochastic,macd,macd_signal,day_of_week,buy_sell
timestamp,,,,,,,,,,,,,,,
2022-09-30 04:30:00,143.3000,143.3000,143.22,143.22,632,143.475008,143.055992,143.282857,143.298101,66.666667,42.553191,-0.028519,-0.035728,4,1
2022-09-30 04:33:00,143.2400,143.2400,143.23,143.23,1121,143.464884,143.059116,143.305000,143.289240,52.459016,10.344828,-0.029383,-0.030800,4,1
2022-09-30 04:36:00,143.2400,143.2400,143.10,143.20,6283,143.464819,143.056181,143.302143,143.277342,46.666667,25.641026,-0.034061,-0.029876,4,1
2022-09-30 04:40:00,143.2100,143.2100,143.20,143.20,5244,143.432500,143.099500,143.250000,143.227923,43.209877,38.297872,-0.048842,-0.036246,4,1
2022-09-30 04:58:00,143.5800,143.5800,143.57,143.57,481,143.570257,143.046743,143.316429,143.378626,68.085106,98.214286,0.048817,-0.001322,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-29 18:58:00,171.2301,171.2301,171.23,171.23,1360,171.664249,171.238291,171.411814,171.388630,26.923077,0.000000,-0.062992,-0.030651,4,1
2023-09-29 18:59:00,171.2500,171.2500,171.24,171.24,300,171.666044,171.207496,171.396814,171.368812,29.411765,3.125000,-0.069098,-0.036208,4,1
2023-09-29 19:13:00,171.2501,171.2501,171.25,171.25,800,171.610306,171.169244,171.353957,171.332361,31.022590,6.250000,-0.072116,-0.055613,4,1


In [15]:
technical_data.to_csv("AAPL_1m_technical.csv")

# Model training

In [215]:
"""Train set"""
TRAIN_SAMPLE_SIZE = int(len(hist) * 0.8)
X = hist[:TRAIN_SAMPLE_SIZE]
Y = hist["is_buy"][:TRAIN_SAMPLE_SIZE]

"""Test set"""
X_test = hist[hist.columns[:-1]][TRAIN_SAMPLE_SIZE-1:-1]
Y_test = hist["is_buy"][TRAIN_SAMPLE_SIZE-1:-1]


print("Training set:")
print(X.shape, Y.shape)

print("\nTesting set:")
print(X_test.shape, Y_test.shape)

Training set:
(688, 18) (688,)

Testing set:
(172, 17) (172,)


In [216]:
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X[X.columns[:-1]])
X_normalized.shape

(688, 17)

In [183]:
X_normalized = X[X.columns[:-1]]

In [217]:
X_scaled = X_normalized
Y_scaled = Y
print(X_scaled.shape, Y_scaled.shape)

(688, 17) (688,)


In [218]:
def perfomance_metric(X_test, Y_test, model):
  Y_preds = model.predict(X_test)
  combined_actual_to_preds = pd.DataFrame(dict(actual=Y_test, prediction=Y_preds))
  print(classification_report(Y_test, Y_preds))
  return pd.crosstab(index=combined_actual_to_preds["actual"], columns=combined_actual_to_preds["prediction"])

In [ ]:
all_normalizer = tf.keras.layers.Normalization(input_shape=(1,))
all_normalizer.adapt(X[X.columns[:-1]])

model = tf.keras.layers.Sequential([
    all_normalizer,
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1),
])

model.compile(optimizer=)

In [219]:
"""Model defination"""
classifier = RandomForestClassifier()
classifier.fit(X_scaled, Y_scaled)
print(perfomance_metric(X_test.values, Y_test.values, model=classifier))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       130
           1       0.36      0.12      0.18        42

    accuracy                           0.73       172
   macro avg       0.56      0.52      0.51       172
weighted avg       0.67      0.73      0.68       172

prediction    0  1
actual            
0           121  9
1            37  5


In [202]:
import numpy as np

def sharpe_ratio(returns, risk_free_rate=0):
    """
    Calculate the Sharpe Ratio.

    Args:
    - returns (np.array or pd.Series): Array or Series of returns.
    - risk_free_rate (float): Risk-free rate, default is 0.

    Returns:
    - float: Sharpe Ratio.
    """
    excess_returns = returns - risk_free_rate
    mean_return = excess_returns.mean()
    std_dev = excess_returns.std()
    sharpe_ratio = mean_return / std_dev if std_dev != 0 else np.nan
    return sharpe_ratio

def value_at_risk(returns, confidence_level=0.95):
    """
    Calculate the Value at Risk (VaR).

    Args:
    - returns (np.array or pd.Series): Array or Series of returns.
    - confidence_level (float): Confidence level, default is 0.95.

    Returns:
    - float: Value at Risk.
    """
    return np.percentile(returns, 100 * (1 - confidence_level))

def maximum_drawdown(returns):
    """
    Calculate the Maximum Drawdown.

    Args:
    - returns (np.array or pd.Series): Array or Series of returns.

    Returns:
    - float: Maximum Drawdown.
    """
    cumulative_returns = np.cumprod(1 + returns)
    peak = cumulative_returns.argmax()
    trough = cumulative_returns[:peak].argmin()
    drawdown = (cumulative_returns[peak] - cumulative_returns[trough]) / cumulative_returns[peak]
    return drawdown

returns = hist['close'].pct_change().dropna()
print(returns)
# Example usage:
print("Sharpe Ratio:", sharpe_ratio(returns))
print("Value at Risk (VaR):", value_at_risk(returns))
print("Maximum Drawdown:", maximum_drawdown(returns))

Date
2014-05-30 00:00:00-04:00   -0.000339
2014-06-02 00:00:00-04:00   -0.010645
2014-06-03 00:00:00-04:00   -0.016229
2014-06-04 00:00:00-04:00   -0.000514
2014-06-05 00:00:00-04:00    0.016965
                               ...   
2024-04-24 00:00:00-04:00    0.007379
2024-04-25 00:00:00-04:00   -0.019553
2024-04-26 00:00:00-04:00    0.099652
2024-04-29 00:00:00-04:00   -0.033335
2024-04-30 00:00:00-04:00   -0.019416
Name: close, Length: 2497, dtype: float64
Sharpe Ratio: 0.04885905527308035
Value at Risk (VaR): -0.02648469313221368
Maximum Drawdown: 0.8585982515250006


/tmp/ipykernel_64502/2518108231.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drawdown = (cumulative_returns[peak] - cumulative_returns[trough]) / cumulative_returns[peak]
